In [1]:
import sys
print(sys.executable)

c:\projetos\FD-TD\.venv\Scripts\python.exe


In [5]:
!pip3 install opencv-python --break-system-packages
!pip3 install pyqt5 --break-system-packages
!pip3 install scipy --break-system-packages

In [2]:
# import pycuda.driver as cuda
# import pycuda.autoinit
# from pycuda.compiler import SourceModule
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

import cv2
import time

from config import *
from util import *
from leapfrog2D import *

ENGINE = 'qt5'

if ENGINE == 'qt5':
    %matplotlib qt5
else:
    %matplotlib widget

In [3]:
#S = configSimulation('laser pulse', anecroic=True)
#S = configSimulation('single lens')
#S = configSimulation('single lens plane')
#S = configSimulation('yagi_antenna')
#S = configSimulation('antireflex')
#S = configSimulation('opticalfiber')
#S = configSimulation('optical ckt')
#S = configSimulation('dual optcal fiber')
#S = configSimulation('refraction')  # mega demo!

# homework simulations
#S = configSimulation('wall right')
#S = configSimulation('wall left')
#S = configSimulation('wall above')
#S = configSimulation('wall below')
#S = configSimulation('room')
#S = configSimulation('closed room')
#S = configSimulation('room with lens')
#S = configSimulation('room with receiver')
S = configSimulation('flat room with receiver')

# config variables
Nt = S['Nt']
dt = S['dt']
Nx = S['Nx']
Ny = S['Ny']

# initial field values
Hx = np.zeros((S['Ny'],S['Nx']))
Hy = np.zeros((S['Ny'],S['Nx']))
Ez = np.zeros((S['Ny'],S['Nx']))

# medium
s = S['s']
er = S['er']
ur = S['ur']
wire = S['wire']
dt = S['dt']

# sources
sources_parameters = S['sources_parameters']
sources_names = S['sources_names']

# FD-TD constants (normalization discretization steps, etc)
[c1, c2, c3, c4] = leapFrog2D_constants(S['er'], S['ur'], S['s'], S['dt']);

parameters = {}
parameters['c1'] = c1;
parameters['c2'] = c2;
parameters['c3'] = c3;
parameters['c4'] = c4;
parameters['dx'] = S['dx']
parameters['dy'] = S['dy']
parameters['wire'] = wire

# colormaps
[M1, M2, M3, M4] = customColormaps();
Imaterial = composeImage(s, wire, er, M2, M3, M4)

# Drawing stuff

fig, ax = plt.subplots(figsize=(10, 8))
ax_img = ax.imshow(Imaterial)
ax.set_aspect('equal')
ax.set_axis_off()

fourcc = cv2.VideoWriter_fourcc(*"X264")
video_out = cv2.VideoWriter('output.mp4', fourcc, 30.0, (Nx,Ny))

# --- Preparação da Mensagem ---

def string_to_bits(s: str) -> list[int]:
    """Converte string para lista de bits (8 bits por char)"""
    bits = []
    for char in s:
        bin_str = f"{ord(char):08b}"
        bits.extend(int(b) for b in bin_str)
    return bits

# 1. Definição da mensagem
MENSAGEM = "Oi"  
bits_msg = string_to_bits(MENSAGEM)

print(f"Mensagem: '{MENSAGEM}'")
print(f"Bits a transmitir: {bits_msg}")
print(f"Total de bits: {len(bits_msg)}")

# Configura a duração de cada bit na simulação
# Nt é o número total de passos de tempo que a simulação tem (ex: 1500)
# Precisamos garantir que a mensagem cabe no tempo da simulação.
steps_per_bit = int(Nt / (len(bits_msg) + 2)) 
# (+2 é uma folga para o sinal não acabar exatamente no último segundo)
print(f"Duração de cada bit: {steps_per_bit} steps")
if steps_per_bit < 20:
    print("ALERTA: O tempo de simulação (Nt) é muito curto para essa mensagem! Aumente S['Nt'] na config.")

# ---------------------------------------



# --- CONFIGURAÇÃO DA TRANSMISSÃO ---
MODULATE_ASK = True   # <--- False não modula o sinal(sem mensagem)
USE_MATCHED_FILTER = True  # <--- OPÇÃO: True = Sinal Limpo; False = Sinal Bruto

MENSAGEM = "Oi"
bits_raw = string_to_bits(MENSAGEM)
bits_str_debug = "".join(str(b) for b in bits_msg)

# PROTOCOLO: Adiciona um 'Start Bit' (1) no começo para acordar o receptor
# e garantir que o primeiro '0' da mensagem seja lido corretamente.
if MODULATE_ASK:
    bits_msg = [1] + bits_raw 
    print(f"Protocolo: Adicionado Start Bit. Total: {len(bits_msg)} bits.")
else:
    bits_msg = bits_raw

# Recalcula steps apenas se a modulação estiver ativa, senão usa 1 (pra não dar erro de divisão por zero)
if MODULATE_ASK:
    # --- CALCULO AUTOMÁTICO DO TEMPO ---
    
    # Define quantos passos de tempo dura CADA BIT. 
    # 120 é um bom número para garantir que a onda senoidal tenha ciclos suficientes (frequência 40)
    steps_per_bit = 120 
    
    # Tempo extra para a onda viajar da fonte até o receptor (Propagation Delay)
    buffer_propagacao = 1000 
    
    # Calcula o novo Nt necessário
    total_steps_necessarios = (len(bits_msg) * steps_per_bit) + buffer_propagacao
    
    # Se o tempo padrão da config for curto, aumentamos ele
    if total_steps_necessarios > Nt:
        print(f"⚠️ AVISO: Aumentando Nt de {Nt} para {total_steps_necessarios} para caber a mensagem.")
        Nt = total_steps_necessarios
    
    print(f"MODO ASK: Enviando '{MENSAGEM}' ({len(bits_msg)} bits).")
    print(f"Duração por bit: {steps_per_bit} steps.")
    print(f"Tempo Total Simulação: {Nt} steps.")
else:
    steps_per_bit = 1 
    print("MODO NORMAL: Sinal contínuo (sem modulação de bits).")

# ---------------------------------------------------------
# CONFIGURAÇÃO DO RECEIVER 
# ---------------------------------------------------------
receiver_data = [] # Lista para guardar o sinal capturado
receiver_data_filtered = [] # Lista para guardar o sinal filtrado (envelope)
rx_pixel = 0
ry_pixel = 0

has_receiver = 'receivers' in S and len(S['receivers']) > 0

if has_receiver:
    # Pega o primeiro receiver da lista
    rec = S['receivers'][0] 
    
    # Converte de 0.0-1.0 para Pixels (Inteiros)
    rx_pixel = int(rec['cx'] * Nx)
    ry_pixel = int(rec['cy'] * Ny)
    
    # Garante que está dentro da tela
    rx_pixel = np.clip(rx_pixel, 0, Nx-1)
    ry_pixel = np.clip(ry_pixel, 0, Ny-1)
    
    print(f"Receiver posicionado em: X={rx_pixel}, Y={ry_pixel}")

# ---------------------------------------------------------

#------ LÓGICA DE RECEPÇÃO (RX) ------
#  Sincronização (Carrier Detect): O receptor fica "dormindo" até perceber que a energia do sinal subiu acima de um nível mínimo (significa que a onda chegou).
# Acumulação: Uma vez acordado, ele guarda as amostras de Ez em um balde (buffer).
# Decisão (Bit Slicer): Quando o balde enche (atinge o tamanho de steps_per_bit), calculamos a média.
#     Média Alta (> limiar) = Bit 1
#     Média Baixa (< limiar) = Bit 0
# Montagem: A cada 8 bits, convertemos para letra e mostramos na tela.

# --- ESTADO DO RECEPTOR ---
rx_buffer = []          
rx_decoded_bits = []    
rx_message_text = ""    
rx_active = False       
rx_silence_counter = 0
rx_timeout_limit = steps_per_bit * 3 # Timeout mais longo

# CONFIGURAÇÕES DO DECODIFICADOR
RX_THRESHOLD_START = 5.0  # Amplitude mínima para "acordar" o receptor (Carrier Detect)
RX_THRESHOLD_BIT = 5.0   # Amplitude média para considerar que é Bit 1 (se for menor, é 0)

# -------------------------------
# Variável para o Filtro (Média Móvel Exponencial)
rx_filtered_val = 0.0
FILTER_ALPHA = 0.2 # Fator de suavização (0.01 a 1.0). Quanto menor, mais suave (mas com mais delay).


# Cria uma string única com todos os bits para exibir na tela (Ex: "01001111...")
bits_str = "".join(str(b) for b in bits_msg)

# main time loop 
# --- VARIÁVEIS DE CONTROLE EXTRA ---
rx_silence_counter = 0      # Conta passos sem sinal para detectar fim
rx_timeout_limit = steps_per_bit * 2 # Se ficar 2 bits em silêncio, considera FIM
rx_raw_bits_display = ""    # String para debug visual dos bits

# main time loop 
t = 0
n = 1

# Variável bits_str_debug aqui para evitar o NameError
bits_str_debug = "".join(str(b) for b in bits_msg)

# Guarda a amplitude original
amplitude_original = sources_parameters[0]['amplitude']

# Inicializa imagem
I = addEzToComposite(Imaterial, Ez/2.0, M1)

while n < Nt:
    n = n + 1
    t = t + dt

    # --- 1. TRANSMISSÃO (TX) ---
    idx_atual = int(n / steps_per_bit)
    
    if MODULATE_ASK:
        if idx_atual < len(bits_msg):
            bit_atual = bits_msg[idx_atual]
            sources_parameters[0]['amplitude'] = amplitude_original if bit_atual == 1 else 0
        else:
            sources_parameters[0]['amplitude'] = 0 

    # --- 2. FÍSICA (FDTD) ---
    for source_parameter, source_name in zip(sources_parameters, sources_names):
        source_parameter['time'] = t;
        Ez += generateSource(Nx, Ny, source_name, source_parameter)
    
    leapFrog2D(Ez, Hy, Hx, parameters)

    # --- 3. RECEPÇÃO (RX) ---
    val_abs = 0 
   
    if has_receiver:
        val = Ez[ry_pixel, rx_pixel]
        val_abs = abs(val) 
        
        receiver_data.append(val)
        receiver_data_filtered.append(rx_filtered_val)
        
        # Filtro
        rx_filtered_val = (rx_filtered_val * (1 - FILTER_ALPHA)) + (val_abs * FILTER_ALPHA)
        signal_processed = rx_filtered_val if USE_MATCHED_FILTER else val_abs

        if not rx_active:
            if signal_processed > RX_THRESHOLD_START:
                rx_active = True
                rx_silence_counter = 0
                
        if rx_active:
            if signal_processed < RX_THRESHOLD_START:
                rx_silence_counter += 1
            else:
                rx_silence_counter = 0 
            
            if rx_silence_counter > rx_timeout_limit:
                rx_active = False 

            rx_buffer.append(signal_processed)
            
            if len(rx_buffer) >= steps_per_bit:
                energia_media = sum(rx_buffer) / len(rx_buffer)
                novo_bit = 1 if energia_media > RX_THRESHOLD_BIT else 0
                rx_decoded_bits.append(novo_bit)
                rx_buffer = [] 

                # --- DECODIFICAÇÃO SMART  ---
                rx_message_text = "" 
                if 1 in rx_decoded_bits:
                    start_index = rx_decoded_bits.index(1)
                    bits_uteis = rx_decoded_bits[start_index+1:]
                    num_bytes = len(bits_uteis) // 8
                    for i in range(num_bytes):
                        byte = bits_uteis[i*8 : (i+1)*8]
                        try:
                            char_code = int("".join(str(b) for b in byte), 2)
                            if 32 <= char_code <= 126:
                                rx_message_text += chr(char_code)
                            else:
                                rx_message_text += "?"
                        except: pass
                                
    # --- 4. VISUALIZAÇÃO COMPACTA (RODAPÉ) ---
    if n % 10 == 0: 
        I = addEzToComposite(Imaterial, Ez/5.0, M1)

        # Receiver Spot
        if has_receiver:
            y_min, y_max = max(0, ry_pixel-4), min(Ny, ry_pixel+4)
            x_min, x_max = max(0, rx_pixel-4), min(Nx, rx_pixel+4)
            cor_rx = [0, 255, 0] if USE_MATCHED_FILTER else [0, 255, 255]
            I[y_min:y_max, x_min:x_max, :] = cor_rx

        # --- CAIXA TX: CANTO INFERIOR ESQUERDO ---
        # Posição Y: Altura Total (Ny) - 100 pixels
        box_h = 90
        box_w = 260
        y_base = Ny - box_h - 10
        x_base = 10
        
        # Fundo Preto e Borda
        cv2.rectangle(I, (x_base, y_base), (x_base + box_w, y_base + box_h), (0, 0, 0), -1) 
        cv2.rectangle(I, (x_base, y_base), (x_base + box_w, y_base + box_h), (255, 255, 255), 1)

        # Texto Menor (Escala 0.5)
        cv2.putText(I, f"TX Msg: {MENSAGEM}", (x_base + 10, y_base + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        if MODULATE_ASK and idx_atual < len(bits_msg):
            b_val = bits_str_debug[idx_atual]
            cor_bit = (0, 255, 0) if b_val == '1' else (0, 0, 255) 
            
            lbl = "START" if idx_atual == 0 else f"BIT {b_val}"
            cv2.putText(I, f"Envio: {lbl} [{idx_atual + 1}/{len(bits_msg)}]", (x_base + 10, y_base + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, cor_bit, 1)
            
            # Bits compactos
            x_b = x_base + 10
            for i, bit_char in enumerate(bits_str_debug):
                if i < idx_atual: c = (100, 100, 100)
                elif i == idx_atual: c = cor_bit
                else: c = (200, 200, 200)
                
                # Mostra janela deslizante de bits para caber na caixinha
                if i >= idx_atual - 2 and i < idx_atual + 12:
                   cv2.putText(I, bit_char, (x_b, y_base + 75), cv2.FONT_HERSHEY_SIMPLEX, 0.5, c, 1)
                   x_b += 12
        else:
            cv2.putText(I, "TX: CONCLUIDO", (x_base + 10, y_base + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 255, 100), 1)
            
        # --- CAIXA RX: CANTO INFERIOR DIREITO ---
        if has_receiver:
            x_rx = Nx - box_w - 10 # Alinha à direita
            
            # Fundo Preto
            cv2.rectangle(I, (x_rx, y_base), (x_rx + box_w, y_base + box_h), (0, 0, 0), -1)
            cv2.rectangle(I, (x_rx, y_base), (x_rx + box_w, y_base + box_h), (255, 255, 255), 1)

            # Dados RX
            cv2.putText(I, f"Sinal: {rx_filtered_val:.1f}", (x_rx + 10, y_base + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
            cor_thr = (0, 255, 0) if rx_filtered_val > RX_THRESHOLD_BIT else (0, 0, 255)
            cv2.putText(I, f"Ref: {RX_THRESHOLD_BIT}", (x_rx + 140, y_base + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, cor_thr, 1)

            # Bits Recebidos
            bits_show = "".join(str(b) for b in rx_decoded_bits)
            if len(bits_show) > 0:
                cv2.putText(I, f"Bits: ...{bits_show[-18:]}", (x_rx + 10, y_base + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
            
            # MENSAGEM FINAL (Destacada)
            cv2.putText(I, f"MSG: {rx_message_text}", (x_rx + 10, y_base + 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        if ENGINE == 'qt5':
            ax_img.set_data(I)
            plt.pause(0.01) 
        else:
            ax_img.set_data(I)
            fig.canvas.draw()
            
    if n % 2 == 0: 
       video_out.write(I[:,:,[2, 1, 0]])

video_out.release()
print("Simulação Concluída.")

# ==============================================================================
# RELATÓRIO FINAL 
# ==============================================================================
print("\n" + "="*40)
print("       RELATÓRIO DE TRANSMISSÃO       ")
print("="*40)

# Constrói string dos bits recebidos
str_rx = "".join(str(b) for b in rx_decoded_bits) 
print(f"Bruto (RX):    {str_rx}")

# Lógica de Decodificação Final (Mesma do loop)
msg_final = ""
if '1' in str_rx:
    idx_start = str_rx.find('1')
    bits_uteis = str_rx[idx_start+1:]
    print(f"Sincronizado:  {bits_uteis}")
    
    num_bytes = len(bits_uteis) // 8
    for i in range(num_bytes):
        byte = bits_uteis[i*8 : (i+1)*8]
        try:
            c = int(byte, 2)
            if 32 <= c <= 126: msg_final += chr(c)
            else: msg_final += "?"
        except: pass

print("-" * 40)
print(f"Mensagem Final: '{msg_final}'")
print("-" * 40)


Mensagem: 'Oi'
Bits a transmitir: [0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1]
Total de bits: 16
Duração de cada bit: 83 steps
Protocolo: Adicionado Start Bit. Total: 17 bits.
⚠️ AVISO: Aumentando Nt de 1500 para 3040 para caber a mensagem.
MODO ASK: Enviando 'Oi' (17 bits).
Duração por bit: 120 steps.
Tempo Total Simulação: 3040 steps.
Receiver posicionado em: X=560, Y=208
Simulação Concluída.

       RELATÓRIO DE TRANSMISSÃO       
Bruto (RX):    1010011110110100100
Sincronizado:  010011110110100100
----------------------------------------
Mensagem Final: 'Oi'
----------------------------------------


In [37]:
# ==============================================================================
# PRÉ-VISUALIZAÇÃO DO SINAL TRANSMISSOR (TX)
# ==============================================================================
import matplotlib.pyplot as plt
import numpy as np

# Verifica se as variáveis de configuração existem
if 'bits_msg' in locals() and 'Nt' in locals():
    
    # 1. Recupera parâmetros da fonte
    freq_tx = sources_parameters[0]['frequency']
    amp_tx = 1000 # Amplitude padrão usada no código
    
    # 2. Cria vetores de tempo e sinal
    t_steps = np.arange(Nt)          # Eixo X (Passos de tempo)
    t_seconds = t_steps * dt         # Tempo em segundos para o cálculo do seno
    tx_signal_ideal = np.zeros(Nt)   # O sinal modulado (Senoide)
    tx_digital_ideal = np.zeros(Nt)  # O envelope digital (Quadrado)
    
    print(f"Gerando prévia do sinal para: '{MENSAGEM}'...")

    # 3. Reconstrói o sinal (Mesma lógica do loop principal)
    for n in range(Nt):
        idx_bit = int(n / steps_per_bit)
        
        if idx_bit < len(bits_msg):
            if bits_msg[idx_bit] == 1:
                # Bit 1: Senoide cheia
                tx_signal_ideal[n] = amp_tx * np.sin(2 * np.pi * freq_tx * t_seconds[n])
                tx_digital_ideal[n] = amp_tx
            else:
                # Bit 0: Silêncio
                tx_signal_ideal[n] = 0
                tx_digital_ideal[n] = 0
    
    # 4. Plota o Gráfico
    plt.figure(figsize=(12, 5))
    
    # Desenha o envelope digital (sombra cinza) para ver onde é 0 e 1
    plt.fill_between(t_steps, tx_digital_ideal, color='gray', alpha=0.2, label='Envelope Digital (Bits)')
    
    # Desenha o sinal modulado (verde)
    plt.plot(t_steps, tx_signal_ideal, color='green', linewidth=1, label='Sinal ASK Modulado (Tx)')
    
    plt.title(f"Check-up do Transmissor: Sinal Esperado para '{MENSAGEM}'")
    plt.xlabel("Tempo (steps)")
    plt.ylabel("Amplitude")
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

else:
    print("ERRO: Rode a célula de Configuração primeiro para definir 'bits_msg' e 'Nt'.")

Gerando prévia do sinal para: 'Oi'...


In [62]:
video_out.release()

In [40]:
plt.imshow(S['s'])
plt.show()

In [41]:
# Plota o sinal capturado
if has_receiver:
    plt.figure(figsize=(10,4))
    plt.plot(receiver_data)
    plt.title(f"Sinal recebido na posição X={rx_pixel}, Y={ry_pixel}")
    plt.xlabel("Tempo (steps)")
    plt.ylabel("Amplitude (Ez)")
    plt.grid(True)
    plt.show()
else:
    print("Nenhum receiver configurado.")

In [32]:
if has_receiver and len(receiver_data) > 0:
    plt.figure(figsize=(12, 6))
    
    # 1. Plota o sinal bruto (Ondas reais) em cinza/azul clarinho
    plt.plot(receiver_data, color='lightgray', label='Sinal Bruto (Ez)')
    
    # 2. Plota o sinal filtrado (Envelope detectado) em vermelho forte
    plt.plot(receiver_data_filtered, color='red', linewidth=2, label='Sinal Filtrado (Envelope)')
    
    # 3. Desenha a linha do Threshold (Limiar) para você ver onde ele corta o 0 e 1
    plt.axhline(y=RX_THRESHOLD_BIT, color='green', linestyle='--', label='Limiar de Decisão (Bit 1)')

    plt.title(f"Recepção ASK: Bruto vs Filtrado (Pos: {rx_pixel}, {ry_pixel})")
    plt.xlabel("Tempo (steps)")
    plt.ylabel("Amplitude")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("Nenhum dado capturado.")